# Keras (MXNet Backend) を利用してMNISTの分類を行う

## 概要

このノートブックでは、Amazon SageMaker上でKerasを実行し、MNISTの手書き数字の分類を学習します。Kerasを動作させるpythonのファイルとして、以下の2種類を`./src`に保存しています。

- [keras_mlp_mnist.py](./src/keras_mlp_mnist.py)  
Kerasで簡単なMLPを構築したものです。
- [keras_cnn_mnist.py](./src/keras_cnn_mnist.py)  
KerasのMNIST用の[CNN](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)をベースに作成しています。


## データセットのS3へのアップロード

- keras.datasetsを利用してmnistのデータをダウンロードしてnpz形式で保存します。
- 保存したnpz形式のファイルを、SageMaker Python SDKを利用してS3にアップロードします。

In [ ]:
import os
import keras
import numpy as np
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/train', image=x_train, label=y_train)
np.savez('./data/test', image=x_test, label=y_test)

import sagemaker
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='dataset/mnist')
print('Training data is uploaded to: {}'.format(input_data))

## コードの変更箇所

kerasのMNIST用の[CNN](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)とSageMaker用のコード[keras_cnn_mnist.py](./src/keras_cnn_mnist.py)の違いを紹介します。SageMaker用のコードでは以下の4点が異なります。


**1. 以下のmain guardの中に、学習してモデルを保存するまでのコードを書くようにします。**

```python
    if __name__ == '__main__':
```

**2. Main guardの最初には、APIから渡される引数やS3上の学習データの位置、モデルの保存場所を受け取るコードを書きます。**

```python
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument('--epochs', type=int, default=10)
    parser.add_argument('--batch-size', type=int, default=100)
    parser.add_argument('--n-class', type=int, default=10)
    ...
    
    args, _ = parser.parse_known_args()
```
**3. 学習が完了すると、KerasのモデルでなくMXNetのモデルとして保存し、後にデプロイで利用します。**

MXNetのモデルは読み込む際に、入力データの名前が必要になるので、最初のレイヤに名前を指定します。仮に`name=data`とすると、名前は`/data_input1`になります。
```python
    model.add(Flatten(input_shape=input_shape, name ='data'))
```
MXNetのモデルで保存し、後でデプロイします。

```python
    keras.models.save_mxnet_model(model=model, prefix=os.path.join(args.model_dir, 'model'))
```

**4. デプロイ用に以下のmodel_fnの関数を定義し、デプロイするモデルを読み込むコードを書きます。**

ここではMXNetのモデルを(Gluonのモデルとして)読み込みます。
```python
  
def model_fn(model_dir):
    import mxnet as mx
    from mxnet import gluon
    net = gluon.nn.SymbolBlock.imports(os.path.join(model_dir, 'model-symbol.json'),
                                   ['/data_input1'], 
                                   param_file=os.path.join(model_dir, 'model-0000.params'),
                                   ctx=mx.cpu())
    return net
```

## SageMakerでの学習

MXNetをバックエンドとして学習するため、MXNetのEstimatorを呼び出します。利用するpythonのコードは、`keras_mlp_mnist.py`と`keras_cnn_mnist.py`のどちらでも可能です。MLPのほうが短時間で学習が終わります。

In [ ]:
from sagemaker.mxnet import MXNet
from sagemaker import get_execution_role

role = get_execution_role()

m = MXNet("./src/keras_mlp_mnist.py",
          role=role,
          train_instance_count=1,
          train_instance_type="ml.m4.xlarge",
          framework_version="1.3.0",
          py_version='py3',
          hyperparameters={'batch-size': 64,
                         'n-class': 10,
                         'epochs': 2})

m.fit(input_data)

## SageMakerでのデプロイ

推論を行うために学習したモデルをデプロイします。deploy() メソッドでは、デプロイ先エンドポイントのインスタンス数、インスタンスタイプを指定します。

In [ ]:
predictor = m.deploy(instance_type='ml.m4.xlarge',
                                   initial_instance_count=1)

デプロイが終わったら実際に手書き文字認識を行ってみましょう。ランダムに5枚選んで推論をしてみます。

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_test.shape[0] - 1), num_samples)
images, labels = x_test[indices]/255, y_test[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = predictor.predict(images.reshape(num_samples, 28, 28, 1))
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))